# Interoperation with ANDES

One of the most interesting feature of AMS is its interoperation with dynamic simulator ANDES.

Interoperation includes compatible case conversion and data exchange, thus it facilitates dispatch-dynamic co-simulation using AMS and ANDES.

In [1]:
import numpy as np

import andes
import ams

import datetime

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'andes:{andes.__version__}')
print(f'ams:{ams.__version__}')

Last run time: 2024-11-24 17:47:02
andes:1.9.2
ams:0.9.12


In [3]:
ams.config_logger(stream_level=20)

## Dispatch

In [4]:
sp = ams.load(ams.get_case('ieee14/ieee14_uced.xlsx'),
              setup=True,
              no_output=True,)

Working directory: "/Users/jinningwang/work/ams/examples"
Parsing input file "/Users/jinningwang/work/miniconda3/envs/amsre/lib/python3.12/site-packages/ams/cases/ieee14/ieee14_uced.xlsx"...
Input file parsed in 0.0261 seconds.
System set up in 0.0016 seconds.


In [5]:
sp.RTED.init()

Building system matrices
Parsing OModel for <RTED>
Evaluating OModel for <RTED>
Finalizing OModel for <RTED>
<RTED> initialized in 0.0106 seconds.


True

In [6]:
sp.RTED.run(solver='CLARABEL')

<RTED> solved as optimal in 0.0115 seconds, converged in 10 iterations with CLARABEL.


True

## Convert to ANDES

The built-in ANDES interface can convert an AMS case to ANDES case in memory.

The bridge between AMS and converted ANDES is the shared power flow devices, Bus, PQ, PV, Slack, Line, and Shunt.

In [7]:
sa = sp.to_andes(setup=True,
                 addfile=andes.get_case('ieee14/ieee14_full.xlsx'))

> Reloaded generated Python code of module "pycode".
Generated code for <ESD1, EV1, EV2> is stale.
Numerical code generation (rapid incremental mode) started...


Generating code for 3 models on 12 processes.


Saved generated pycode to "/Users/jinningwang/.andes/pycode"
> Reloaded generated Python code of module "pycode".
Generated numerical code for 3 models in 0.4299 seconds.
Parsing additional file "/Users/jinningwang/work/miniconda3/envs/amsre/lib/python3.12/site-packages/andes/cases/ieee14/ieee14_full.xlsx"...
Following PFlow models in addfile will be overwritten: <Bus>, <PQ>, <PV>, <Slack>, <Shunt>, <Line>, <Area>
Addfile parsed in 0.0278 seconds.
System converted to ANDES in 0.5277 seconds.
AMS system 0x32344ab70 is linked to the ANDES system 0x32344a060.
System internal structure set up in 0.0179 seconds.
> Reloaded generated Python code of module "pycode".
System internal structure set up in 0.0140 seconds.
Parsing OModel for <PFlow>
Evaluating OModel for <PFlow>
Finalizing OModel for <PFlow>
-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           

If you wish to add devices to the converted ANDES system, set `setup=False` to skip the ANDES setup process.

As indicated by the output information, in the conversion process, ANDES power flow devices will be overwritten by AMS ones, if exists.

Upon a successful conversion, you are ready to enjoy full capability of ANDES.

``help`` command can give a quick reference.

In [8]:
help(sp.to_andes)

Help on method to_andes in module ams.system:

to_andes(setup=True, addfile=None, **kwargs) method of ams.system.System instance
    Convert the AMS system to an ANDES system.

    A preferred dynamic system file to be added has following features:
    1. The file contains both power flow and dynamic models.
    2. The file can run in ANDES natively.
    3. Power flow models are in the same shape as the AMS system.
    4. Dynamic models, if any, are in the same shape as the AMS system.

    Parameters
    ----------
    setup : bool, optional
        Whether to call `setup()` after the conversion. Default is True.
    addfile : str, optional
        The additional file to be converted to ANDES dynamic mdoels.
    **kwargs : dict
        Keyword arguments to be passed to `andes.system.System`.

    Returns
    -------
    andes : andes.system.System
        The converted ANDES system.

    Examples
    --------
    >>> import ams
    >>> import andes
    >>> sp = ams.load(ams.get_case('

## Interoperation with ANDES

In the interface class ``dyn``, the link table is stored in ``dyn.link``.

It describes the mapping relationships between power flow devices and dynamic devices.

In [9]:
sp.dyn.link

,stg_idx,bus_idx,syg_idx,gov_idx,dg_idx,rg_idx,gammap,gammaq
0,Slack_1,1,GENROU_1,TGOV1_1,NaN,NaN,1.0,1.0
1,PV_5,8,GENROU_5,TGOV1_5,NaN,NaN,1.0,1.0
2,PV_4,6,GENROU_4,TGOV1_4,NaN,NaN,1.0,1.0
3,PV_3,3,GENROU_3,TGOV1_3,NaN,NaN,1.0,1.0
4,PV_2,2,GENROU_2,TGOV1_2,NaN,NaN,1.0,1.0


### Send

As there is a gap between DC-based dispatch and AC-based TDS, a conversion is required to ensure the TDS initialization.

In [10]:
sp.RTED.dc2ac()

Parsing OModel for <ACOPF>
Evaluating OModel for <ACOPF>
Finalizing OModel for <ACOPF>
<ACOPF> initialized in 0.0032 seconds.
<ACOPF> solved in 0.1401 seconds, converged in 12 iterations with PYPOWER-PIPS.
Parsing OModel for <RTED>
<RTED> converted to AC.


True

In the RTED routine, there are two mapping dictionaries to define the data exchange, namely, `map1` for receiving data from ANDES and `map2` for sending data to ANDES.

In [11]:
sp.RTED.map2

OrderedDict([('vBus', ('Bus', 'v0')),
             ('ug', ('StaticGen', 'u')),
             ('pg', ('StaticGen', 'p0'))])

In [12]:
sp.dyn.send(adsys=sa, routine='RTED')

Send <RTED> results to ANDES <0x32344a060>...
*Send <vBus> to StaticGen.v0
Send <vBus> to Bus.v0
Send <ug> to StaticGen.u
Send <pg> to StaticGen.p0


True

### Run ANDES

Sometimes, the ANDES TDS initialization may fail due to inapproriate limits.

Here, we alleviate the `TGOV1` limit issue by enlarging the `Pmax` and `Pmin` to the same value.

In [13]:
sa.TGOV1.alter(src='VMAX', idx=sa.TGOV1.idx.v, value=100*np.ones(sa.TGOV1.n))
sa.TGOV1.alter(src='VMIN', idx=sa.TGOV1.idx.v, value=np.zeros(sa.TGOV1.n))

Run power flow.

In [14]:
sa.PFlow.run()

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0080 seconds.
0: |F(x)| = 0.7743935696
1: |F(x)| = 0.01847784692
2: |F(x)| = 3.493405927e-05
3: |F(x)| = 1.193747323e-10
Converged in 4 iterations in 0.0125 seconds.


True

Try to init TDS.

In [15]:
_ = sa.TDS.init()

Initialization for dynamics completed in 0.0198 seconds.
Initialization was successful.


Run TDS.

In [16]:
sa.TDS.config.no_tqdm = True # disable progress bar
sa.TDS.run()


-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0.0-20.0 s.
Fixed step size: h=33.33 ms. Shrink if not converged.
Simulation to t=20.00 sec completed in 0.2986 seconds.


True

### Receive

In [17]:
sp.RTED.map1

OrderedDict([('ug', ('StaticGen', 'u')), ('pg0', ('StaticGen', 'p'))])

In [18]:
sp.dyn.receive(adsys=sa, routine='RTED')

Receive <ug> from SynGen.u
Receive <pg0> from SynGen.Pe


True

The RTED parameter ``pg0``, is retrieved from ANDES as the corresponding generator output power.

In [19]:
sp.RTED.pg0.v

array([1.79503641, 0.48417982, 0.01000094, 0.02000094, 0.01000095])